<a href="https://colab.research.google.com/github/tamnhu2913/Applied-Text-Mining-in-Python/blob/main/assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not.

In [2]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('assets/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [3]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'],
                                                    spam_data['target'],
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [4]:
def answer_one():
  return spam_data['target'].value_counts(normalize=True)[1]*100

answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
  vect = CountVectorizer().fit(X_train)
  return max(vect.get_feature_names_out(), key = len)

answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
  vect = CountVectorizer().fit(X_train)
  X_train_vec = vect.transform(X_train)
  X_test_vec = vect.transform(X_test)
  model = MultinomialNB(alpha=0.1).fit(X_train_vec, y_train)
  pred = model.predict_proba(X_test_vec)[:,1]

  return roc_auc_score(y_test, pred)

answer_three()

0.991545422134696

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters. The transformed data will be a compressed sparse row matrix where the number of rows is the number of documents in `X_train`, the number of columns is the number of features found by the vectorizer in each document, and each value in the sparse matrix is the tf-idf value. First find the **max** tf-idf value for every feature.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf among the **max** tf-idf values?

Put these features in two series where each series is sorted by tf-idf value. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. Any entries with identical tf-ids should appear in lexigraphically increasing order by their feature name in boh series. For example, if the features "a", "b", "c" had the tf-idfs 1.0, 0.5, 1.0 in the series with the largest tf-idfs, then they should occur in the returned result in the order "a", "c", "b" with values 1.0, 1.0, 0.5.

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
  vect = TfidfVectorizer().fit(X_train)
  feature_names = np.array(vect.get_feature_names_out())

  X_train_vec = vect.transform(X_train)
  sorted_tfidf_index = X_train_vec.max(0).toarray()[0].argsort()

  df_tfidf = pd.DataFrame({'feature': feature_names, 'max': X_train_vec.max(0).toarray()[0]})
  large = df_tfidf.tail(20).sort_values(by='max', ascending=False)
  small = df_tfidf.head(20).sort_values(by='max', ascending=True)
  large = large.set_index('feature')['max']
  small = small.set_index('feature')['max']
  return (small, large)

answer_four()

(feature
 01223585334     0.224162
 000pes          0.228624
 00              0.237314
 0089            0.242052
 04              0.245530
 02              0.251159
 0207            0.260485
 008704050406    0.260735
 021             0.265662
 03              0.274527
 0430            0.275955
 02072069400     0.280990
 02085076972     0.281621
 01223585236     0.284874
 02073162414     0.288430
 0121            0.302440
 000             0.361385
 05              0.461408
 0125698789      0.501280
 0578            0.520041
 Name: max, dtype: float64,
 feature
 ì_          0.656928
 û_thanks    0.566275
 ìï          0.562825
 û_          0.475710
 åè10        0.444475
 ûò          0.362962
 zeros       0.360084
 ûówell      0.348800
 zindgi      0.321114
 zoe         0.300639
 ó_          0.288038
 ûï          0.284009
 ûïharry     0.278142
 zouk        0.270640
 zhong       0.268083
 åòit        0.264900
 åð          0.257006
 åôrents     0.228624
 ì¼1         0.220940
 zyada       0.1

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [8]:
def answer_five():
  vect = TfidfVectorizer(min_df=3).fit(X_train)
  X_train_vec = vect.transform(X_train)
  X_test_vec = vect.transform(X_test)

  model = MultinomialNB(alpha=0.1).fit(X_train_vec, y_train)
  pred = model.predict_proba(X_test_vec)[:,1]
  return roc_auc_score(y_test, pred)

answer_five()

0.9954968337775665

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [9]:
def answer_six():
  data = spam_data.copy()
  data['length'] = data['text'].apply(len)
  avg = data.groupby('target')['length'].mean().to_list()
  return tuple(avg)

answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [10]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*Hint: Since probability is set to false, use the model's `decision_function` on the test data when calculating the target scores to use in roc_auc_score*

*This function should return the AUC score as a float.*

In [11]:
from sklearn.svm import SVC

def answer_seven():
  vect = TfidfVectorizer(min_df=5).fit(X_train)
  X_train_vec = vect.transform(X_train)
  X_test_vec = vect.transform(X_test)

  X_train_len = [len(s) for s in X_train]
  X_test_len = [len(s) for s in X_test]

  X_train_final = add_feature(X_train_vec, X_train_len)
  X_test_final = add_feature(X_test_vec, X_test_len)

  model = SVC(C=10000).fit(X_train_final, y_train)
  pred = model.decision_function(X_test_final)

  return roc_auc_score(y_test, pred)

answer_seven()

0.9963202213809143

### Question 8

What is the average number of digits per document for not spam and spam documents?

*Hint: Use `\d` for digit class*

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [12]:
def answer_eight():
  data = spam_data.copy()
  data['len_digit'] = data['text'].str.findall(r'\d').str.len()
  avg = data.groupby('target')['len_digit'].mean().tolist()
  return tuple(avg)

answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100` and `max_iter=1000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [13]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
  import re
  vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
  X_train_vec = vect.transform(X_train)
  X_test_vec = vect.transform(X_test)

  def len_doc(X):
    return [len(s) for s in X]
  def len_digit(X):
    return [len(re.findall(r'\d', s)) for s in X]

  X_train_features = add_feature(X_train_vec, [len_doc(X_train), len_digit(X_train)])
  X_test_features = add_feature(X_test_vec, [len_doc(X_test), len_digit(X_test)])

  model = LogisticRegression(C=100, max_iter=1000).fit(X_train_features, y_train)
  pred = model.predict_proba(X_test_features)[:,1]
  return roc_auc_score(y_test, pred)

answer_nine()

0.9975298371899564

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [14]:
def answer_ten():
  data = spam_data.copy()
  data['len_digit'] = data['text'].str.findall(r'\W').str.len()
  avg = data.groupby('target')['len_digit'].mean().tolist()
  return tuple(avg)

answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the **first 2000 rows** of training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100 and max_iter=1000. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [15]:
def answer_eleven():
  import re
  vect = CountVectorizer(min_df=5, ngram_range=(2,5),analyzer='char_wb').fit(X_train[:2000])
  X_train_vec = vect.transform(X_train[:2000])
  X_test_vec = vect.transform(X_test)

  def len_doc(X):
    return [len(s) for s in X]
  def len_digit(X):
    return [len(re.findall(r'\d', s)) for s in X]
  def len_non_word(X):
    return [len(re.findall(r'\W', s)) for s in X]
  X_train_features = add_feature(X_train_vec, [len_doc(X_train[:2000]),
                                               len_digit(X_train[:2000]), len_non_word(X_train[:2000])])
  X_test_features = add_feature(X_test_vec, [len_doc(X_test),
                                             len_digit(X_test), len_non_word(X_test)])

  model = LogisticRegression(C=100, max_iter=1000).fit(X_train_features, y_train[:2000])
  pred = model.predict_proba(X_test_features)[:,1]

  feature_names = np.concatenate([np.array(vect.get_feature_names_out()), ['length_of_doc', 'digit_count', 'non_word_char_count']])
  sorted_coef_index = model.coef_[0].argsort()
  small_coef = feature_names[sorted_coef_index[:10]]
  large_coef = feature_names[sorted_coef_index[:-11:-1]]
  return (roc_auc_score(y_test, pred), small_coef.tolist(), large_coef.tolist())

answer_eleven()


(0.9931964416073884,
 ['. ', '..', ' i', 'at', ' m', 'n ', 'he', 'i ', 'ok', '? '],
 ['digit_count', 'co', 'ne', 'xt', ' st', 'xt ', 's ', 'es', ' f', 'on'])